GAT implementation (node classification) - Cora Dataset

https://arxiv.org/pdf/1710.10903 (GAT Research Paper)

In [1]:
# dataset
from torch_geometric.datasets import Planetoid

# torch libraries
import torch
import torch.nn.functional as F
from torch_geometric.nn import GATConv
from torch.optim import Adam 
from torch.nn import Linear, Dropout

In [2]:
def gather_data():
    # get the Cora dataset
    """CORA DATASET:
        - the Cora datset is a large graph network that shows 2708 scientific publications
        our goal is to classify it into 1 of 7 classes
    """
    dataset = Planetoid(root=".", name="Cora")
    cora = dataset[0]

    print(cora)

    return dataset, cora

In [3]:
dataset, data = gather_data()

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])


In [4]:
# GAT class
"""METHODS:
    __init__ : initializes my custom GAT model
    forward : the defined forward pass method for my GAT model
    train : training loop that prints out the model loss per 10th epoch
    test : testing loop that returns the final model accuracy
"""
class CoraGAT(torch.nn.Module):
    def __init__(self, input, hidden, output, heads=8):
        """PARAMETERS:
            input : the number of features for each node in the graph
            hidden : size of the hidden layer (needs to be tuned) -> defined as hidden*heads
            output : the number of output features
            heads : defined to be 8, number of attention heads (each head learns a different type of attention pattern)
        """
        super(CoraGAT, self).__init__()

        # first GAT layer with 8 attention heads
        self.conv1 = GATConv(
            input,
            hidden,
            heads=heads,
            dropout=0.6
        )

        # second GAT layer with 1 attention head
        self.conv2 = GATConv(
            hidden*heads,
            output,
            heads=1,
            concat=False,
            dropout=0.6
        )

    def forward(self, x, edge_index):
        # first GAT layer
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv1(x, edge_index)
        x = F.elu(x)

        # second GAT layer
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)
    
    # analysis methods
    def train_model(self, data, epochs):
        # initialize the optimizer
        optimizer = Adam(self.parameters(), lr=0.005)

        self.train()

        for i in range(epochs):
            # get model prediction 
            out = self(data.x, data.edge_index)

            # calculate loss amount
            loss_fn = F.nll_loss(out[data.train_mask], data.y[data.train_mask])

            # backpropagate through the GAT
            loss_fn.backward()
            optimizer.step()

            # reset the gradients
            optimizer.zero_grad()

            ###### PRINTOUTS EVERYE 10th EPOCH
            if i%10==0:
                print(f"Epochs: {i}/{epochs}\tLoss:{loss_fn.item()}")
        

    @torch.no_grad()
    def test_model(self, data):
        self.eval()

        out = self(data.x, data.edge_index)
        pred = out.argmax(dim=1)
        correct = pred[data.test_mask] == data.y[data.test_mask]
        accuracy = int(correct.sum()) / int(data.test_mask.sum())

        return accuracy


In [5]:
# initialize the model
gat = CoraGAT(dataset.num_features, hidden=8, output=dataset.num_classes)

In [6]:
# print out training loop
gat.train_model(data, 1000)

Epochs: 0/1000	Loss:1.9963725805282593
Epochs: 10/1000	Loss:1.2424739599227905
Epochs: 20/1000	Loss:0.8960310816764832
Epochs: 30/1000	Loss:0.7475935816764832
Epochs: 40/1000	Loss:0.6328566670417786
Epochs: 50/1000	Loss:0.6520478129386902
Epochs: 60/1000	Loss:0.5875628590583801
Epochs: 70/1000	Loss:0.5228483080863953
Epochs: 80/1000	Loss:0.45635709166526794
Epochs: 90/1000	Loss:0.4738793969154358
Epochs: 100/1000	Loss:0.33258935809135437
Epochs: 110/1000	Loss:0.3398807942867279
Epochs: 120/1000	Loss:0.453147292137146
Epochs: 130/1000	Loss:0.25478512048721313
Epochs: 140/1000	Loss:0.4987379312515259
Epochs: 150/1000	Loss:0.4391951560974121
Epochs: 160/1000	Loss:0.4094718396663666
Epochs: 170/1000	Loss:0.2776276469230652
Epochs: 180/1000	Loss:0.3151956796646118
Epochs: 190/1000	Loss:0.2404175102710724
Epochs: 200/1000	Loss:0.37491241097450256
Epochs: 210/1000	Loss:0.3914780020713806
Epochs: 220/1000	Loss:0.3592813014984131
Epochs: 230/1000	Loss:0.2518371045589447
Epochs: 240/1000	Loss:0.

In [7]:
# print out testing loop
acc = gat.test_model(data)
print(f"GAT Model Accuracy: {acc}")

GAT Model Accuracy: 0.767
